In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
#load and preprocess data
train_df = pd.read_csv(train_file_path, sep='\t',header=None,names=['label','text'])
test_df = pd.read_csv(test_file_path,sep='\t',header=None,names=['label','text'])

#encode labels: ham =0, spam=1
label_map = {'ham':0,'spam':1}
train_df['label'] = train_df['label'].map(label_map)
test_df['label'] = test_df['label'].map(label_map)

#define TextVector layer
max_vocab_size = 1000
sequence_length = 100

vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=max_vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length
)

#adapt vectorizer to training text
vectorizer.adapt(train_df['text'].values)

#prepare dataset
x_train = vectorizer(train_df['text'].values)
y_train = train_df['label'].values

x_test = vectorizer(test_df['text'].values)
y_test = test_df['label'].values

In [ ]:
#build model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(sequence_length,)),
    tf.keras.layers.Embedding(max_vocab_size, 64),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

#model commpile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#training model
model.fit(x_train, y_train,
          epochs=20,
          validation_data=(x_test, y_test),
          verbose=2
          )

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  process_text = vectorizer([pred_text])
  prediction = model.predict(process_text,verbose=0)[0][0]
  label='spam' if prediction >0.5 else 'ham'
  return [float(prediction),label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
